# Cleanup 

**Caution!** Below code will clean-up all data set groups and associated objects in Amazon Personalize. Execute multiple times if required! 

In [ ]:
%matplotlib inline

import boto3, os
import json
import time
import sagemaker

personalize = boto3.client('personalize')
os.environ['AWS_DEFAULT_REGION']="us-east-1"


In [ ]:
import time
def delete_solution(arn):
    response = personalize.delete_solution(
        solutionArn=arn
    )
    print("Delete solution:" + str(response) + '\n')
    time.sleep(30)

def delete_campaign(arn):
    response = personalize.delete_campaign(
        campaignArn=arn
    )
    print("Delete Campaign:" + str(response) + '\n')
    time.sleep(30)
    
def delete_dataset(arn):
    response = personalize.delete_dataset(
        datasetArn=arn
    )
    print("Delete Dataset:" + str(response) + '\n')
    time.sleep(30)
    
def delete_dataset_group(arn):
    response = personalize.list_solutions(
        datasetGroupArn=arn
    )
    for solution in response['solutions']:
        response = personalize.list_campaigns(
            solutionArn=solution['solutionArn']
        )
        for campaign in response['campaigns']:
            delete_campaign(campaign['campaignArn'])
        delete_solution(solution['solutionArn'])
    
    response = personalize.list_datasets(
        datasetGroupArn=arn
    )
    for dataset in response['datasets']:
        delete_dataset(dataset['datasetArn'])
    
    response = personalize.delete_dataset_group(
        datasetGroupArn=arn
    )
    print("Delete Dataset Group:" + str(response) + '\n')   
    

In [ ]:
response = personalize.list_dataset_groups(
)
if len(response['datasetGroups']) == 0:
    print('Success all Dataset Groups and associated artifacts where deleted')
else:
    for dataSetGroup in response['datasetGroups']:
        delete_dataset_group( dataSetGroup['datasetGroupArn'])
    
    

## Deleting Schemas
Schemas are a little tricky since they are not defined within the UI of Amazon Personalize or as part of a Data Set Group. The cell below will iterate over each of the schemas and delete those no longer in use.

In [ ]:
schemas_resp = personalize.list_schemas()['schemas']
for item in schemas_resp:
    try:
        personalize.delete_schema(schemaArn=item['schemaArn'])
    except Exception:
        pass